# SQLiteを操作する用のクラスの定義

In [1]:
import time
import sqlite3

class StopWatch:
    '''
    処理時間計測
    '''
    def sw_start(self):
      self.__starttime = time.time()
      return
    
    def sw_stop(self):
      print("■処理時間：")
      print(f"{time.time() - self.__starttime}[sec]")
      return 

class Sqlite3Access:
  # 必要に応じて変更して下さい
  __colab_dir = "/content/drive/MyDrive/Colab Notebooks/"
  
  def __init__(self, _db_name):
    self.db_name = _db_name
    self.sw = StopWatch()
    return
  
  def _connect(self):
    '''
    すでに存在すればコネクションをはる
    存在しなければ新たに作成する
    '''
    self.conn = sqlite3.connect(self.__colab_dir + self.db_name + ".db")
    return

  def _getCursor(self):
    '''
    sqliteを操作するためのカーソルを取得する
    '''
    self._connect()
    return self.conn.cursor()

  def select(self, _sql):
    '''
    select
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()
    try:
      # データ検索
      cur.execute(_sql)
    except Exception as e:
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return null
    
    # 大量データの取り出しを考慮してカーソルのままretrunする
    return cur

  def execute(self, _sql) -> bool:
    '''
    create
    insert
    updaet
    delete
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()

    try:
      self.sw.sw_start()
      
      # データ登録
      cur.execute(_sql)

      # コミットしないと登録が反映されない
      self.conn.commit()
    except Exception as e:
      self.sw.sw_stop()
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return False
    
    self.sw.sw_stop()
    self.conn.close()
    return True 

  def executemany(self, _list, _sql):
    '''
    create
    insert
    updaet
    delete
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()

    try:
      self.sw.sw_start()
      
      # 複数データ登録
      cur.executemany(_sql, _list)

      # コミットしないと登録が反映されない
      self.conn.commit()
    except Exception as e:
      self.sw.sw_stop()
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return False
    self.sw.sw_stop()
    self.conn.close()
    return True
  
  def printSelect(self, _sql):
    '''
    select結果をコンソールに出力する
    '''
    print("■実行SQL：")
    print(_sql)
    print("■実行結果：")
    self.sw.sw_start()
    for row in self.select(_sql):
        print(row)
    self.sw.sw_stop()

# Googleドライブのマウント

In [2]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# メイン処理

## 初期化

In [3]:
test = Sqlite3Access("test2")
test.db_name

'test2'

## テーブルの作成

In [4]:
# create文を作成
_sql =  """\
CREATE TABLE testtable(id INTEGER PRIMARY KEY AUTOINCREMENT, key STRING, val STRING)
"""

# create文を実行
test.execute(_sql)

■処理時間：
0.02193760871887207[sec]


True

## データの登録（1レコード）

In [5]:
# insert文を作成
_sql =  """\
INSERT INTO testtable(key,val) values('あ行', 'あいうえお')
"""

test.execute(_sql)

■処理時間：
0.0117340087890625[sec]


True

## データの取得（1レコード）

In [6]:
# select文を作成
_sql = """\
select * 
from testtable
"""

test.printSelect(_sql)

■実行SQL：
select * 
from testtable

■実行結果：
(1, 'あ行', 'あいうえお')
■処理時間：
0.00792551040649414[sec]


## データの登録（複数レコード）

In [7]:
# insert文を使用して複数レコードを同時に登録する
inserts = []
inserts.append(("か行","かきくけこ"))
inserts.append(("さ行","さしすせそ"))
inserts.append(("た行","たちつてと"))

test.executemany(inserts, 'INSERT INTO testtable(key,val) values(?, ?)')

■処理時間：
0.012566566467285156[sec]


True

## データの取得（件数指定）

In [8]:
# select文を作成、先頭3レコードのみを取得
_sql = """\
select * 
from testtable
-- 先頭3レコードのみを取得
limit 3
"""

test.printSelect(_sql)

■実行SQL：
select * 
from testtable
-- 先頭3レコードのみを取得
limit 3

■実行結果：
(1, 'あ行', 'あいうえお')
(2, 'か行', 'かきくけこ')
(3, 'さ行', 'さしすせそ')
■処理時間：
0.003291606903076172[sec]
